In [5]:
import mercury as mr
!pip install playwright-stealth


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [6]:
app = mr.App(title="A Spotify Scraper", description ="let's pray mercury works")

mercury.App

In [28]:
artistInput = mr.Text(label = "Artist Name:", value="Caravan Palace",rows = 1)

mercury.Text

In [29]:
mr.Md(f"""## Okay, let's test {artistInput.value}!""")

## Okay, let's test Caravan Palace!

In [32]:
import pandas as pd
import spotipy
import os
from dotenv import load_dotenv

load_dotenv()

project_api_key=os.environ['project_api_key']
project_api_key_secret= os.environ['project_api_key_secret']

from spotipy.oauth2 import SpotifyClientCredentials

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
     project_api_key,
     project_api_key_secret,
))

results = spotify.search(q='artist: '+ artistInput.value, type='artist')
uri = pd.Series([results['artists']['items'][0]['uri']])
uri = uri.str.replace('spotify:artist:', '')

#print(uri)

url = "https://open.spotify.com/artist/"+uri[0]
#print(url)

from playwright.async_api import async_playwright
from playwright_stealth import stealth_async

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = True)
page = await browser.new_page()

await page.goto(url)

all_Tay = page.locator('See more')
await page.click('text=See more')

from bs4 import BeautifulSoup

doc = BeautifulSoup(await page.content())

top_songs = doc.find_all('div', role='row')

rows = []
rank = 1
artists = []
title = doc.find_all(class_ = "encore-text encore-text-headline-large encore-internal-color-text-base")
top_songs = doc.find_all('div', role='row')
for song in top_songs:
    #print("----")

    row = {}

    try:
        row['title'] = song.find(class_="encore-text encore-text-body-medium encore-internal-color-text-base btE2c3IKaOXZ4VNAb8WQ standalone-ellipsis-one-line").text
    except:
        row['title'] = None

    try:
         row['artist'] = title[0].text
    except:
         row['artist'] = None

    try:
        row['play_count'] = song.find(class_="encore-text encore-text-body-small HxDMwNr5oCxTOyqt85gi").text
    except:
        row['play_count'] = None

    rows.append(row)

dataTime = pd.DataFrame(rows)

dataTime['play_count'] = dataTime['play_count'].str.replace(",","")
dataTime['play_count'] = dataTime['play_count'].astype(int)
dataTime = dataTime.sort_values(by = 'play_count', ascending = False)
dataTime['rank'] = [1,2,3,4,5,6,7,8,9,10]
dataTime.set_index('rank', inplace = True)

percentage = (dataTime['play_count'][1]/sum(dataTime['play_count']))*100

if percentage < 85:
    # print(f"Your artist, {artistInput.value}, is NOT a one hit wonder!")
    # print(f"Their top song, {dataTime['title'][1]}, made up {percentage}% of their total listens of {sum(dataTime['play_count'])} at {dataTime['play_count'][1]}")
    yesOrNo = "IS NOT"
else:
    # print(f"Your artist, {artistInput.value}, IS a one hit wonder!")
    # print(f"Their top song, {dataTime['title'][1]}, made up {percentage}% of their total listens of {sum(dataTime['play_count'])} at {dataTime['play_count'][1]}")
    yesOrNo = "IS"

#dataTime

In [33]:
mr.Md(f"""## Your artist, {artistInput.value}, {yesOrNo} a one hit wonder!""")

## Your artist, Caravan Palace, IS NOT a one hit wonder!

In [38]:
mr.Md(f"""## Their top song, {dataTime['title'][1]}, made up {percentage}% of their total listens of {sum(dataTime['play_count'])} at {dataTime['play_count'][1]}""")

## Their top song, Lone Digger, made up 38.073824868133826% of their total listens of 514146831 at 195755364